In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('', os.pardir)))
from utils.misc import read_json_file

analysis_dir = os.path.abspath(os.path.join('', os.pardir))
root_dir = os.path.abspath(os.path.join(analysis_dir, os.pardir))
root_dir

'/home/lorenzo/Desktop/research-projects/mimic-2'

In [2]:
# Select policy
policy = "adaptive_greedy_threshold"

# Select trace
trace = "att.lte.driving"

# Get config
log_dir_path = os.path.join(root_dir, 'log')
mab_dir_path = os.path.join(log_dir_path, 'mab', 'config')
config_file_name = policy + '.json'
file_path = os.path.join(mab_dir_path, config_file_name)
config = read_json_file(file_path)

# Get model timestamp
model_name = config["models"][0]['name']
model_ts = '.'.join(model_name.split('.')[1:])
model_ts

'2023.03.22.14.39.27'

In [3]:
# Get iperf config
exp_ts = config['traces'][0]['timestamp']
iperf_test_dir_path = os.path.join(log_dir_path, 'iperf')
iperf_file_name = trace + '.' + policy + '.' + model_ts + '.json'
iperf_config = read_json_file(os.path.join(iperf_test_dir_path ,iperf_file_name))

In [4]:
client_metrics = iperf_config['end']['streams'][0]['sender']
min_rtt = client_metrics['min_rtt']
thruput = client_metrics['bits_per_second']
min_rtt*10**(-6), thruput

(0.058490999999999994, 786998.628989)